In [40]:
import pandas as pd
import numpy as np

In [41]:
matchup = pd.read_csv("matchup.txt", header=None)
matchup = str(matchup[0].values[0])
print(matchup)


BOS vs. LAC


In [42]:
team1

'BOS'

In [43]:
team1 = matchup.split(" ")[0]
team2 = matchup.split(" ")[-1]


# scrape from basketball reference
team1_stats = pd.read_html(f"https://www.basketball-reference.com/teams/{team1}/2023_games.html")
team2_stats = pd.read_html(f"https://www.basketball-reference.com/teams/{team2}/2023_games.html")

team1_stats = team1_stats[0]
team2_stats = team2_stats[0]

In [44]:
team1_stats.dropna(subset={"Tm"}, inplace=True) # null scores haven't happened yet
team1_stats.drop(columns={"Unnamed: 3", "Unnamed: 4", "Unnamed: 8", "Notes"}, inplace=True) # drop columns we don't need
team1_stats = team1_stats[team1_stats["Date"] != "Date"] # remove header row showing up as a game
team1_stats.rename(columns={"Unnamed: 5": "Home", "Unnamed: 7": "Win"}, inplace=True)

# do above steps for team2
team2_stats.dropna(subset={"Tm"}, inplace=True)
team2_stats.drop(columns={"Unnamed: 3", "Unnamed: 4", "Unnamed: 8", "Notes"}, inplace=True)
team2_stats = team2_stats[team2_stats["Date"] != "Date"]
team2_stats.rename(columns={"Unnamed: 5": "Home", "Unnamed: 7": "Win"}, inplace=True)

In [46]:
# function to convert Home column to 0 or 1
def home_to_binary(home):
    if home == "@":
        return 0
    else:
        return 1

# function to convert Win column to 0 or 1
def win_to_binary(win):
    if win == "W":
        return 1
    else:
        return 0

# function to return day of week (first 3 letters) from date
def get_day_of_week(date):
    return date[:3]

team1_stats["Home"] = team1_stats.apply(lambda x: home_to_binary(x["Home"]), axis=1)
team1_stats["Win"] = team1_stats.apply(lambda x: win_to_binary(x["Win"]), axis=1)
team1_stats["Day"] = team1_stats.apply(lambda x: get_day_of_week(x["Date"]), axis=1)

# set data types
team1_stats["Tm"] = team1_stats["Tm"].astype(int)
team1_stats["Opp"] = team1_stats["Opp"].astype(int)
team1_stats["W"] = team1_stats["W"].astype(int)
team1_stats["L"] = team1_stats["L"].astype(int)

# do same for team2
team2_stats["Home"] = team2_stats.apply(lambda x: home_to_binary(x["Home"]), axis=1)
team2_stats["Win"] = team2_stats.apply(lambda x: win_to_binary(x["Win"]), axis=1)
team2_stats["Day"] = team2_stats.apply(lambda x: get_day_of_week(x["Date"]), axis=1)

team2_stats["Tm"] = team2_stats["Tm"].astype(int)
team2_stats["Opp"] = team2_stats["Opp"].astype(int)
team2_stats["W"] = team2_stats["W"].astype(int)
team2_stats["L"] = team2_stats["L"].astype(int)

In [47]:
team1_stats

,G,Date,Start (ET),Home,Opponent,Win,Tm,Opp,W,L,Streak,Day
0,1,"Tue, Oct 18, 2022",7:30p,1,Philadelphia 76ers,1,126,117,1,0,W 1,Tue
1,2,"Fri, Oct 21, 2022",7:30p,0,Miami Heat,1,111,104,2,0,W 2,Fri
2,3,"Sat, Oct 22, 2022",7:00p,0,Orlando Magic,1,126,120,3,0,W 3,Sat
3,4,"Mon, Oct 24, 2022",8:00p,0,Chicago Bulls,0,102,120,3,1,L 1,Mon
4,5,"Fri, Oct 28, 2022",7:30p,1,Cleveland Cavaliers,0,123,132,3,2,L 2,Fri
5,6,"Sun, Oct 30, 2022",6:00p,1,Washington Wizards,1,112,94,4,2,W 1,Sun
6,7,"Wed, Nov 2, 2022",7:30p,0,Cleveland Cavaliers,0,113,114,4,3,L 1,Wed
7,8,"Fri, Nov 4, 2022",7:30p,1,Chicago Bulls,1,123,119,5,3,W 1,Fri
8,9,"Sat, Nov 5, 2022",7:30p,0,New York Knicks,1,133,118,6,3,W 2,Sat
9,10,"Mon, Nov 7, 2022",9:00p,0,Memphis Grizzlies,1,109,106,7,3,W 3,Mon
